### Challenge 25: Break "random access read/write" AES CTR

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

<div class="alert alert-block alert-info">

Back to CTR. Encrypt the recovered plaintext from this file (the ECB exercise) under CTR with a random key (for this exercise the key should be unknown to you, but hold on to it).

Now, write the code that allows you to "seek" into the ciphertext, decrypt, and re-encrypt with different plaintext. Expose this as a function, like, "edit(ciphertext, key, offset, newtext)".

Imagine the "edit" function was exposed to attackers by means of an API call that didn't reveal the key or the original plaintext; the attacker has the ciphertext and controls the offset and "new text".

Recover the original plaintext.

<div class="alert alert-block alert-warning">
    
### Food for thought.

A folkloric supposed benefit of CTR mode is the ability to easily "seek forward" into the ciphertext; to access byte N of the ciphertext, all you need to be able to do is generate byte N of the keystream. Imagine if you'd relied on that advice to, say, encrypt a disk.

</div>
</div>

In [11]:
from random import randint
from Crypto.Cipher import AES
import cryptopals as cp
import base64

---
First, we'll decrypt the plaintext using the known key (see exercise #7). This is not part of the attack, it's just the setup.

If we actually knew the plaintext, we would be able to trivially find the keystream by XOR'ing the PT and CT together, which isn't the point of this exercise

In [12]:
f = open('./challenge-data/25.txt', 'r')
ciphertext = base64.b64decode(f.read())
f.close()

In [13]:
key = "YELLOW SUBMARINE"
my_aes = AES.new(key, AES.MODE_ECB)
plaintext = my_aes.decrypt(ciphertext)

Now...generate an "unknown" random key and use it to AES-CTR encrypt the plaintext

In [14]:
unknown_key = bytes([randint(0, 256) for __ in range(16)])
ciphertext = cp.AESEncrypt(plaintext, unknown_key, mode='CTR')

---
#### The attack

First, here's the function the challenge tells us to write. It allows us to replace a byte of plaintext at some offset with one of our choosing.

In [15]:
def edit_AES_CTR(ciphertext, key, offset, newtext, IV=[0]*16):

    plaintext = bytearray(cp.AESDecrypt(ciphertext, key, 'CTR', IV))
    plaintext[offset:offset+len(newtext)] = newtext
    ciphertext = cp.AESEncrypt(plaintext, key, 'CTR', IV)

    return ciphertext

The way I solved it was to write a 0x00 as the plaintext to each byte.  Since CTR mode generates a key-stream and simply XORs the plaintext with the corresponding byte of key-stream, this means the resulting ciphertext for the byte we are editing will now = the actual key-stream.  Repeating this for each byte allows us to learn the entire keystream.

In [17]:
key_stream = []
for byte_idx in range(len(ciphertext)):

    ct_new = edit_AES_CTR(ciphertext, unknown_key, byte_idx, [0])
    key_stream.append(ct_new[byte_idx])

Now, to recover the plaintext, we just XOR the recovered key-stream with the original ciphertext.

In [20]:
plaintext = cp.bitwise_xor(key_stream, ciphertext)
print(plaintext.decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


**Note:**  If we were doing this against hard drive encryption, we could restore each byte of the original data after we learn the key-stream byte value.

---
The above attack should work for an unknown fixed, random IV as well...

Let's demonstrate.

Again..generate an "unknown" random key and use it to CTR encrypt the plaintext.  This time, add a random IV.

In [21]:
unknown_key = bytes([randint(0, 256) for __ in range(16)])
unknown_IV = bytes([randint(0, 256) for __ in range(8)])

ciphertext = cp.AESEncrypt(plaintext, unknown_key, 'CTR', unknown_IV)

actual_key_stream = cp.bitwise_xor(plaintext, ciphertext)
key_stream = []

for byte_idx in range(len(ciphertext)):

    ct_new = edit_AES_CTR(ciphertext, unknown_key, byte_idx, [0], unknown_IV)
    key_stream.append(ct_new[byte_idx])

plaintext = cp.bitwise_xor(key_stream, ciphertext)
print(plaintext.decode())

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)